In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim) # 먼저 .log_softmax 함수를 통해 log softmax를 구함 (나중에 cross entropy loss를 계산하기 위함)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1)) # α/K
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) # scatter_ 함수를 통해 target의 index에 해당하는 위치에 (1−α)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim)) # Log softmax와 target을 곱한 것의 음수를 취한 것이 cross entrophy loss가 됨


# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
# [assert 조건] 이라 적었을 때 조건을 충족하지 않는다면 에러를 내라 할 때 사용
# if/else문이나 try/except문처럼 조건에 해당하지 않는 경우에 대응하지 않는 이유는 '에러가 절대 나지 않는다는 확신'을 갖고 있지만 일단 저것이 맞는지 검증하기 위한 용도로 사용하기 때문임
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2   
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1) # softmax를 통해 함수에 들어오는 값들을 0~1의 확률값으로 바꿈

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32) #실제값 T, 예측값 P
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32) #실제값 T, 예측값 N
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32) #실제값 F, 예측값 P
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32) #실제값 F, 예측값 N

        precision = tp / (tp + fp + self.epsilon) #정밀도(모델이 True라고 예측한 정답 중에서 실제로 True인 비율)
        recall = tp / (tp + fn + self.epsilon) #재현율(실제 데이터가 True인 것 중에서 모델이 True라고 예측한 비율)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon) # 모든 요소를 [min, max]범위로 고정하여 Tensor로 출력
        return 1 - f1.mean()


In [2]:

_criterion_entrypoints = {
    'cross_entropy': nn.CrossEntropyLoss,
    'focal': FocalLoss,
    'label_smoothing': LabelSmoothingLoss,
    'f1': F1Loss
}


def criterion_entrypoint(criterion_name):
    return _criterion_entrypoints[criterion_name]


def is_criterion(criterion_name):
    return criterion_name in _criterion_entrypoints

# **kwargs는 (키워드 = 특정 값) 형태로 함수를 호출할 수 있음. 결과값이 딕셔너리형태로 출력됨. 함수를 만들 때 키워드 인수는 가장 마지막으로 가야 함
def create_criterion(criterion_name, **kwargs):
    if is_criterion(criterion_name):
        create_fn = criterion_entrypoint(criterion_name)
        criterion = create_fn(**kwargs)
    else:
        raise RuntimeError('Unknown loss (%s)' % criterion_name)
    return criterion
